# Calibration Plot Generation Notebook

In [ ]:
import pydsm
from pydsm import postpro

import pydelmod
from pydelmod import calibplot

## Load locations from a .csv file 
The .csv file should have atleast 'Name','BPart' and 'Description' columns

In [ ]:
locationfile='e:/dsm2v82_calibration/LocationInfo/calibration_ec_stations.csv'
dfloc = postpro.load_location_file(locationfile)
locations=[postpro.Location(r['Name'],r['BPart'],r['Description']) for i,r in dfloc.iterrows()]
           

## Define variable type 
(e.g. FLOW, STAGE, EC, TEMP, etc i.e. the C Part of the DSS files)

In [ ]:
vartype=postpro.VarType('EC','mmhos/cm')

## Define studies
The studies are a set of a name and dss file that contains the data

In [ ]:
obs_study=postpro.Study('Observed','e:/dsm2v82_calibration/ObservedData/updated_sep-oct2020/cdec_ec_merged_sep2020.dss')

base_dir='e:/dsm2v82_calibration/'
model_output_dir='output/'
# No "." allowed in study names because of issue https://github.com/holoviz/holoviews/issues/4714
study_file_map = {'DCD': base_dir+'BaselineRun/DSM2_DCD/'+model_output_dir+'historical_v81_DCD.dss',
                  'DCD12 SMCD': base_dir+'BaselineRun/DSM2_DCD1.2_SMCD/'+model_output_dir+'historical_v82b1.dss',
                  'DCD12 SMCD_last3': base_dir+'CalibrationRuns/last/DSM2_DCD1.2_SMCD_last3bin/'+model_output_dir+'historical_v82b1.dss',
                  'DICU': base_dir+'BaselineRun/DSM2_DICU/'+model_output_dir+'historical_v81_DICU.dss'}
model_studies=[postpro.Study(name,study_file_map[name]) for name in study_file_map]

studies=[obs_study]+model_studies

## Define timewindow (optional)
Time window in the format of start_date_str - end_date_str. Specify empty string if the available data should decide the time window

In [ ]:
#timewindow='01OCT2013 - 01DEC2013'
timewindow=""

In [ ]:
def build_and_save_plot(studies, location, vartype, timewindow):
    flow_or_stage = (vartype.name == 'FLOW') or (vartype.name == 'STAGE')
    calib_plot_template=calibplot.build_calib_plot_template(studies, location, vartype, timewindow, tidal_template=flow_or_stage)
    calib_plot_template.save(f'{location.name}_{vartype.name}.html')

In [ ]:
import dask
from dask.distributed import Client, LocalCluster

class DaskCluster:
    def __init__(self):
        self.client=None
    def start_local_cluster(self):
        cluster = LocalCluster(n_workers=8, threads_per_worker=1, memory_limit='4G') # threads_per_worker=1 needed if using numba :(
        self.client = Client(cluster)
    def stop_local_cluster(self):
        self.client.shutdown()
        self.client=None

In [ ]:
cluster = DaskCluster()
cluster.start_local_cluster()
cluster.client

In [ ]:
tasks=[dask.delayed(build_and_save_plot)(studies, location, vartype, timewindow, dask_key_name=f'{location.name}/{vartype.name}')
           for location in locations]

In [ ]:
dask.compute(tasks)

In [ ]:
cluster.stop_local_cluster()